In [1]:
import xlrd, xlwt
import requests
import time
from random import randint
from time import sleep
from xlwt import Workbook

In [2]:
excelPath = ("FullDatabase04222020_XLSX.xlsx")

In [3]:
workbook = xlrd.open_workbook(excelPath) # read workbook
sheet = workbook.sheet_by_index(0)

In [4]:
firstRow = []
for i in range(sheet.ncols):
    firstRow.append(sheet.cell_value(0, i))
    
# Filter DOI to be usable
Title, Authors, Journal, DOI = [], [], [], []
for i in range(0, sheet.nrows):
    if sheet.cell_type(i, 10) != xlrd.XL_CELL_EMPTY:
        # filter DOI to be usable
        DOIValue = sheet.cell_value(i, 10)
        DOIOnly = DOIValue.replace("https://doi.org/", "")
        DOIOnly = DOIOnly.replace("http://dx.doi.org/", "")
        DOIOnly = DOIOnly.replace("doi:", "")
        DOIOnly = DOIOnly.split(' ', 1)[0]
        if DOIOnly.startswith("DOI") or DOIOnly.startswith("10"):
            DOI.append(DOIOnly)
            Title.append(sheet.cell_value(i, 0))
            Authors.append(sheet.cell_value(i, 1))
            Journal.append(sheet.cell_value(i, 5))   

In [5]:
wb = Workbook() # write workbook 
filteredSheetForWriting = wb.add_sheet("Filtered Sheet")

In [6]:
# write filtered data
for i in range(len(DOI)):
    filteredSheetForWriting.write(i, 0, Title[i])
    filteredSheetForWriting.write(i, 1, Authors[i])
    filteredSheetForWriting.write(i, 2, Journal[i])
    filteredSheetForWriting.write(i, 3, DOI[i])

In [7]:
wb.save("FilteredDatabase.xls")

In [8]:
# Identify Author's affiliation
# https://api.crossref.org/v1/works/
filteredWorkbook = xlrd.open_workbook("FilteredDatabase.xls")
filteredSheet = filteredWorkbook.sheet_by_index(0)
baseURL = "https://api.crossref.org/v1/works/"

In [9]:
URLS = []
for i in range(1, filteredSheet.nrows):
    URLS.append(baseURL + filteredSheet.cell_value(i,3))

In [10]:
# Retrieve JSON Info
pageInfo =[]
initialTime = time.time()
for i in range(len(URLS)):
    startTime = time.time()
    sleep(randint(0,5))
    page = requests.get(URLS[i])
    if page.status_code == 200:
        pageJson = page.json()
        pageInfo.append(pageJson)
    else:
        pageInfo.append("Response not 200")

In [11]:
# indexing off by two for excel
for i in range(len(pageInfo)):
    if pageInfo[i] == "Response not 200":
        filteredSheetForWriting.write(i+1, 5, "Response not 200")
    elif "author" in pageInfo[i].get("message"):
        if str(pageInfo[i].get("message").get("author")[0].get("affiliation")) == "[]":
            filteredSheetForWriting.write(i+1, 5, "No affiliation found")
        else:
            filteredSheetForWriting.write(i+1, 5, str(pageInfo[i].get("message").get("author")[0].get("affiliation")[0].get("name")))
    else:
        filteredSheetForWriting.write(i+1, 5, "Author Not Found")

In [12]:
wb.save("FilteredDatabase.xls")